## Needed packages

In [1]:
! pip install emot --upgrade

Requirement already up-to-date: emot in /usr/local/lib/python3.7/dist-packages (2.1)


In [2]:
! pip install fasttext

In [3]:
! wget -O /tmp/lid.176.bin https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.bin

--2021-05-04 06:25:16--  https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.bin
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 172.67.9.4, 104.22.75.142, 104.22.74.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|172.67.9.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 131266198 (125M) [application/octet-stream]
Saving to: ‘/tmp/lid.176.bin’

/tmp/lid.176.bin    100%[===================>] 125.18M  12.3MB/s    in 25s     

2021-05-04 06:25:43 (4.94 MB/s) - ‘/tmp/lid.176.bin’ saved [131266198/131266198]



In [4]:
! pip install pycountry

In [5]:
import fasttext

PRETRAINED_MODEL_PATH = '/tmp/lid.176.bin'
language_model = fasttext.load_model(PRETRAINED_MODEL_PATH)

In [6]:
# enable file saving in colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
import tweepy
consumer_key = "ZlqvIhMvShvge0fQt70srSuAk"
consumer_secret = "wFxylCPZW6KR6onyZLfLK5AUlc6vu8Y56EbEtpp6oOk50Y4Lwp"
access_token = "1365107282338607109-7ko2pElkXOod8ilSTKrP4fx5ElpRQG"
access_token_secret = "d671pqtMTij6LRsqwWGA3BMTKTc3jaTyogRTpLUVji8gJ"
auth = tweepy.AppAuthHandler(consumer_key, consumer_secret)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

import numpy as np
import pandas as pd
import re
import nltk
import spacy
import string
import csv
import json
from collections import defaultdict
from emot.emo_unicode import UNICODE_EMO, EMOTICONS
import nltk
nltk.download('all')
from textblob import TextBlob
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import	WordNetLemmatizer
from nltk.tokenize import word_tokenize

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Package cess_esp is already up-to-date!
[nltk_data]    | Downloading packag

## Wiki data demo

In [8]:
# data directory
wiki_data_dir = '/content/drive/My Drive/wiki_data.csv'
wiki_data_df = pd.read_csv(wiki_data_dir)
wiki_data_df.columns =['index', 'Name', 'Age', 'UserName']

In [9]:
wiki_data_df.head(5)

,index,Name,Age,UserName
0,1,Justin Bieber,27.0,justinbieber
1,2,Taylor Swift,31.0,taylorswift13
2,3,Rihanna,33.0,rihanna
3,4,Justin Timberlake,40.0,jtimberlake
4,5,Ellen DeGeneres,63.0,TheEllenShow


In [10]:
credential_list = wiki_data_df['UserName'].tolist()
credential_list[0:5]

['justinbieber', 'taylorswift13', 'rihanna', 'jtimberlake', 'TheEllenShow']

In [11]:
# dict that shows ages and their corresponding accounts
age_dict = dict()

with open('/content/drive/My Drive/wiki_data.csv', mode='r') as wiki_csv:
    reader = csv.reader(wiki_csv, delimiter=',')
    for row in reader:
        age = row[2]
        handle = row[3]
        if age is not '' and int(age)>0:
          age_dict[handle] = age

In [12]:
len(age_dict),age_dict

(688,
 {'10Ronaldinho': '41',
  '143redangel': '36',
  '1victorvaldes': '39',
  '21LVA': '35',
  '3gerardpique': '34',
  'AHMTKURAL': '38',
  'ANAMARIABRAGA': '72',
  'ActuallyNPH': '47',
  'AdalRamones': '59',
  'AdamSchefter': '54',
  'Adela_Micha': '57',
  'Adele': '32',
  'Akon': '48',
  'AlanCarr': '44',
  'AlejandroSanz': '52',
  'AlfredoFlores': '32',
  'AlvaroMorata': '28',
  'Alyssa_Milano': '48',
  'Anahi': '37',
  'AndreaLegarreta': '49',
  'AndreaSernaTV': '44',
  'AnnaKendrick47': '35',
  'AnselElgort': '27',
  'AnupamPkher': '66',
  'AnushkaSharma': '33',
  'ArianaGrande': '27',
  'AshBenzo': '31',
  'Ashton5SOS': '26',
  'Asli_Jacqueline': '35',
  'AudrinaPatridge': '35',
  'AustinMahone': '25',
  'BDUTT': '49',
  'BSchweinsteiger': '36',
  'BeingSalmanKhan': '55',
  'Bellaudyaa': '33',
  'Benzema': '33',
  'Beyonce': '39',
  'BigSean': '33',
  'BillCosby': '83',
  'BillGates': '65',
  'BillNye': '65',
  'BillSimmons': '51',
  'Bourdain': '64',
  'BradPaisley': '48',
  '

## Tweets collecting demo

In [13]:
# extract the latest 200 tweets using api.user_timeline

tweets = api.user_timeline(screen_name='katyperry', 
                           # 200 is the maximum allowed count
                           count=200,
                           include_rts = False,
                           # Necessary to keep full_text 
                           # otherwise only the first 140 words are extracted
                           tweet_mode = 'extended'
                           )

# show the extracted 3 latest tweets
print(len(tweets))

for info in tweets[:3]:
     print("ID: {}".format(info.id))
     print(info.created_at)
     print(info.full_text)
     
     print("\n")

200
ID: 1389021973284491264
2021-05-03 01:00:19
TINK fast - halfway through the Top 10 of #AmericanIdol. Tune in to @ABCNetwork if you believe in fairies as much as I believe in this group #clapclap ✨ https://t.co/TNmYZCRTLv


ID: 1388978234868441089
2021-05-02 22:06:31
legolas’s gf flyin in hot with a pouch full of pixie dust to throw on anyone who doesn’t believe we have the best top 10 in @americanidol history #disneynight ok ✨ LIVE AT 5PST/8EST COAST 2 COAST LEZ GO✨ https://t.co/mz4l6zhvdU


ID: 1388221862027567105
2021-04-30 20:00:58
Just posted a photo https://t.co/UKOSGP5aKQ




## text preparation functions

convert collected tweet strings

In [14]:
# lower casing
def lower_case(text):
  return text.lower()

# Function to remove url's
def remove_urls(text):
  url_pattern = re.compile(r'https?://\S+|www\.\S+')
  return url_pattern.sub(r'', text)

# Function to remove emoji.
def remove_emoji(text):
  emoji_pattern = re.compile("["
                          u"\U0001F600-\U0001F64F"  # emoticons
                          u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                          u"\U0001F680-\U0001F6FF"  # transport & map symbols
                          u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                          u"\U00002702-\U000027B0"
                          u"\U000024C2-\U0001F251"
                          "]+", flags=re.UNICODE)
  return emoji_pattern.sub(r'', text)

# Function for removing emoticons
def remove_emoticons(text):
  emoticon_pattern = re.compile(u'(' + u'|'.join(k for k in EMOTICONS) + u')')
  return emoticon_pattern.sub(r'', text).strip()


def tweet_str_prep(tweet):
  tweet = lower_case(tweet)
  tweet = remove_urls(tweet)
  tweet = remove_emoji(tweet)
  tweet = remove_emoticons(tweet)
  return tweet


convert the tweet strings to list of words

In [15]:
# convert str to words list
def to_word_list (text):
  tweet_list = text.split(' ')
  return tweet_list

# remove @, #, whitespace, punctuation, nonalphabetic, stop words, spelling correction, Stemming, Lemmatization
def word_list_prep(tweet):
  tweet_list = to_word_list (tweet)
  
  while("" in tweet_list) :
      tweet_list.remove("")

  prefixes = ('@','#')
  tweet_list = [x for x in tweet_list if not x.startswith(prefixes)]
  tweet_list = [x.strip(string.punctuation) for x in tweet_list]
  cleaned_tweet = ' '.join(tweet_list)
  tokens = word_tokenize(cleaned_tweet)
  tweet_list = [x.strip(string.punctuation) for x in tokens]
  # remove all tokens that are not alphabetic
  tweet_list = [word for word in tweet_list if word.isalpha()]

  # stop words, spelling correction, Stemming, Lemmatization
  stop_words = set(stopwords.words('english'))
  tweet_list = [w for w in tweet_list if not w in stop_words]
  # print(tweet_list)
  for word in tweet_list:
    corrected = str(TextBlob(word).correct())
    stemmed = PorterStemmer().stem(corrected)
    lemmatized = WordNetLemmatizer().lemmatize(stemmed)

    tweet_list = [sub.replace(word, lemmatized) for sub in tweet_list]
    return tweet_list

function demo

In [16]:
tweet = tweets[5].full_text
tweet = tweet_str_prep(tweet)
tweet_list = word_list_prep(tweet)
tweet, tweet_list

('a tuna behind the scenes of our very sofishticated @resortsworldlv  splash',
 ['tune', 'behind', 'scenes', 'sofishticated', 'splash'])

## Tweets Processing

In [17]:
word_dict = dict()
count_dict = dict()
accounts_count = len(age_dict)
for count, handle in enumerate(age_dict):
  print(count+1, handle)
  try:
    tweets = api.user_timeline(screen_name=handle, 
                            # 200 is the maximum allowed count
                            count=200,
                            include_rts = False,
                            # Necessary to keep full_text 
                            # otherwise only the first 140 words are extracted
                            tweet_mode = 'extended'
                            )
    word_dict[handle] = defaultdict(int)
    tweets_count = len(tweets)
    for tweet in tweets:
      tweet = tweet.full_text
      try:
        # language prediction
        prediction = language_model.predict(tweet)
        if '__label__en' == prediction[0][0]:
          tweet = tweet_str_prep(tweet)
          word_list = word_list_prep(tweet)
          for word in word_list:
            word_dict[handle][word] += 1
        else:
          tweets_count -= 1
      except:
        tweets_count -= 1
        # print (handle, tweet)
    count_dict[handle] = tweets_count  
    # break
  except:
    accounts_count -= 1
    print('tweets can not be collected for ' + handle + ', there are %d accounts left.' % accounts_count)
# word_dict, count_dict

1 katyperry
2 justinbieber
3 taylorswift13
4 rihanna
5 jtimberlake
6 TheEllenShow
7 britneyspears
8 Cristiano
9 KimKardashian
10 shakira
11 selenagomez
12 JLo
13 ArianaGrande
14 ddlovato
15 jimmyfallon
16 Oprah
17 Drake
18 BillGates
19 KingJames
20 BrunoMars
21 Adele
22 NiallOfficial
23 MileyCyrus
24 aliciakeys
25 KevinHart4real
26 Louis_Tomlinson
27 NICKIMINAJ
28 neymarjr
29 Eminem
30 EmWatson
tweets can not be collected for EmWatson, there are 687 accounts left.
31 ActuallyNPH
32 danieltosh
33 davidguetta
34 SrBachchan
35 aplusk
36 khloekardashian
37 kourtneykardash
38 iamsrk
39 edsheeran
40 xtina
41 JimCarrey
42 kanyewest
43 chrisbrown
44 aamir_khan
tweets can not be collected for aamir_khan, there are 686 accounts left.
45 BeingSalmanKhan
46 Beyonce
47 blakeshelton
48 RyanSeacrest
49 ivetesangalo
50 LeoDiCaprio
51 ParisHilton
52 iamwill
53 KendallJenner
54 tyrabanks
55 SnoopDogg
56 AlejandroSanz
57 ricky_martin
58 10Ronaldinho
59 SimonCowell
60 WayneRooney
61 KDTrey5
62 radityadika

In [18]:
word_vector_json = json.dumps(word_dict)
print(len(word_dict))
word_vector_f = open("/content/drive/My Drive/word_dict.json","w")
word_vector_f.write(word_vector_json)
word_vector_f.close()

654


In [19]:
tweet_count_json = json.dumps(count_dict)
print(len(count_dict))
tweet_count_f = open("/content/drive/My Drive/tweet_count.json","w")
tweet_count_f.write(tweet_count_json)
tweet_count_f.close()

654


In [31]:
word_dict['katyperry']

defaultdict(int,
            {'across': 1,
             'actual': 6,
             'actually': 2,
             'add': 1,
             'advice': 2,
             'age': 1,
             'aight': 1,
             'aka': 1,
             'alanadelsherm': 1,
             'alanissophia': 1,
             'alien': 1,
             'already': 2,
             'alway': 2,
             'always': 2,
             'america': 5,
             'americanidol': 2,
             'amirite': 1,
             'amp': 1,
             'announce': 1,
             'another': 2,
             'answering': 1,
             'anymore': 2,
             'anyon': 2,
             'anyone': 2,
             'anyway': 1,
             'apple': 1,
             'ar': 1,
             'arguabl': 1,
             'argument': 1,
             'artists': 1,
             'ask': 1,
             'atcha': 1,
             'audience': 1,
             'audition': 2,
             'auditions': 2,
             'away': 1,
             'babies': 1,
      